# Задание 1. Прямые методы решения линейных систем

In [34]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [35]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [arg for arg in argv] + ['$$'])
    display(Markdown(res))

def get_latex_column(arg: str, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {arg} with indeces form 1 to {n}"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def print_system_output(A: Matrix, B: Matrix, X: Matrix, n: int) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(r'A \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


In [36]:
n = int(input()) # Размерность матрицы A
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_tex('A = ', A.tex(), ',~~', 'B = ', B.tex())
print_tex(A.tex(), get_latex_column('x', n), '=', B.tex())

$$ A =  \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 1.928219 & 6.992132 & 6.386559 & 5.102684 \\ 1.827392 & 2.611487 & 9.924339 & 3.280103 \\ 1.818859 & 8.605652 & 0.065562 & 6.859980 \end{pmatrix} ,~~ B =  \begin{pmatrix} 1.691335 \\ 7.379943 \\ 5.782934 \\ 3.799967 \end{pmatrix} $$

$$ \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 1.928219 & 6.992132 & 6.386559 & 5.102684 \\ 1.827392 & 2.611487 & 9.924339 & 3.280103 \\ 1.818859 & 8.605652 & 0.065562 & 6.859980 \end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}
\end{pmatrix} = \begin{pmatrix} 1.691335 \\ 7.379943 \\ 5.782934 \\ 3.799967 \end{pmatrix} $$

### Решение методом Гаусса единственного деления

In [37]:
def SingleGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    A_ = A.copy()
    B_ = B.copy()
    X = [[0] for _ in range(n)]
    for k in range(n):
        for j in range(k+1, n):
            d = A_[j][k] / A_[k][k]
            for i in range(k, n):
                A_[j][i] = A_[j][i] - d * A_[k][i]
            B_[j] -= d * B_[k][0]
    for k in range(n-1, -1, -1):
        d = 0
        for j in range(k, n):
            s = A_[k][j] * X[j][0]
            d += s
        X[k][0] = (B_[k][0] - d) / A_[k][k]
    return Matrix(np.array(X))


print_system_output(A, B, SingleGauss(A, B, n), n)


$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} -3.007789 \\ 1.221092 \\ 0.877614 \\ -0.188794 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 1.928219 & 6.992132 & 6.386559 & 5.102684 \\ 1.827392 & 2.611487 & 9.924339 & 3.280103 \\ 1.818859 & 8.605652 & 0.065562 & 6.859980 \end{pmatrix} \begin{pmatrix} -3.007789 \\ 1.221092 \\ 0.877614 \\ -0.188794 \end{pmatrix} = \begin{pmatrix} 1.691335 \\ 7.379943 \\ 5.782934 \\ 3.799967 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
1.6913346217988412\\
7.379942983393452\\
5.782934368203914\\
3.7999666293549836
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
1.6913346217988412\\
7.379942983393451\\
5.782934368203914\\
3.7999666293549845
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
8.881784197001252e-16\\
0.0\\
-8.881784197001252e-16
\end{pmatrix} $$

### Решение методом Гаусса с выбором главного элемента

In [38]:
def ModifiedGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        p = k # главный элемент
        for m in range(k+1, n):
            if abs(AB[p][k]) < abs(AB[m][k]):
                p = m # индекс строки, содержащей максимальный элемент в столбце
        AB.inverse_rows(k, p) # перестановка строк
        for m in range(k+1, n):
            c = AB[m][k] / AB[k][k]
            for i in range(n+1):
                AB[m][i] -= c * AB[k][i]
        X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
        for k in range(n-1, -1, -1):
            s = 0
            for i in range(k+1, n):
                s += AB[k][i] * X[i][0]
            X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

print_system_output(A, B, ModifiedGauss(A, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} -3.007789 \\ 1.221092 \\ 0.877614 \\ -0.188794 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 1.928219 & 6.992132 & 6.386559 & 5.102684 \\ 1.827392 & 2.611487 & 9.924339 & 3.280103 \\ 1.818859 & 8.605652 & 0.065562 & 6.859980 \end{pmatrix} \begin{pmatrix} -3.007789 \\ 1.221092 \\ 0.877614 \\ -0.188794 \end{pmatrix} = \begin{pmatrix} 1.691335 \\ 7.379943 \\ 5.782934 \\ 3.799967 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
1.6913346217988412\\
7.379942983393452\\
5.782934368203914\\
3.7999666293549836
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
1.691334621798843\\
7.379942983393453\\
5.7829343682039145\\
3.7999666293549836
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
-1.7763568394002505e-15\\
-8.881784197001252e-16\\
-8.881784197001252e-16\\
0.0
\end{pmatrix} $$

### Решение методом $LU$-разложения

In [39]:
def GetLU(A: Matrix, n: int) -> Tuple[Matrix, Matrix]:
    L = Matrix(np.zeros(shape=(n, n)))
    U = A.copy()
    for i in range(n):
        for j in range(i, n):
            L[j][i] = U[j][i] / U[i][i]
    for k in range(1, n):
        for i in range(k-1, n):
            for j in range(i, n):
                L[j][i] = U[j][i] / U[i][i]
        for i in range(k, n):
            for j in range(k-1, n):
                U[i][j] = U[i][j] - L[i][k-1] * U[k-1][j]
    return L, U

L, U = GetLU(A, n)
print_tex('L = ', L.tex())
print_tex('U = ', U.tex())
print_tex(r'L \times U = ', (np.matmul(L, U)).tex(), r'\stackrel{?}{=}', A.tex(), '= A')

$$ L =  \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0.369223 & 1 & 0 & 0 \\ 0.349916 & -0.082752 & 1 & 0 \\ 0.348282 & 1.459175 & -1.051674 & 1 \end{pmatrix} $$

$$ U =  \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 0 & 3.896336 & 3.323205 & 4.866757 \\ 0 & 0 & 7.296168 & 3.459247 \\ 0 & 0 & 0 & 3.173983 \end{pmatrix} $$

$$ L \times U =  \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 1.928219 & 6.992132 & 6.386559 & 5.102684 \\ 1.827392 & 2.611487 & 9.924339 & 3.280103 \\ 1.818859 & 8.605652 & 0.065562 & 6.859980 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 1.928219 & 6.992132 & 6.386559 & 5.102684 \\ 1.827392 & 2.611487 & 9.924339 & 3.280103 \\ 1.818859 & 8.605652 & 0.065562 & 6.859980 \end{pmatrix} = A $$

In [40]:
Y_ = SingleGauss(L, B, n)
X_ = SingleGauss(U, Y_, n)
print_system_output(A, B, X_, n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} -3.007789 \\ 1.221092 \\ 0.877614 \\ -0.188794 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 5.222372 & 8.384629 & 8.296766 & 0.638982 \\ 1.928219 & 6.992132 & 6.386559 & 5.102684 \\ 1.827392 & 2.611487 & 9.924339 & 3.280103 \\ 1.818859 & 8.605652 & 0.065562 & 6.859980 \end{pmatrix} \begin{pmatrix} -3.007789 \\ 1.221092 \\ 0.877614 \\ -0.188794 \end{pmatrix} = \begin{pmatrix} 1.691335 \\ 7.379943 \\ 5.782934 \\ 3.799967 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
1.6913346217988412\\
7.379942983393452\\
5.782934368203914\\
3.7999666293549836
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
1.6913346217988412\\
7.379942983393453\\
5.782934368203913\\
3.7999666293549845
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
-8.881784197001252e-16\\
8.881784197001252e-16\\
-8.881784197001252e-16
\end{pmatrix} $$

### Нахождение обратной к матрице $A$

In [41]:
def inverse(A: Matrix, n: int) -> Matrix:
    R = Matrix(np.zeros(shape=(n, 1)))
    A_ = Matrix(np.zeros(shape=(n, n)))
    for i in range(n):
        R[i][0] = 1.
        S = SingleGauss(A, R, n)
        for j in range(n):
            A_[j][i] = S[j][0]
        R[i][0] = 0
    return A_

A_ = inverse(A, n)
print_tex(r'A^{-1} = ', A_.tex())
print_tex(r'A \times A^{-1} = ', (np.matmul(A, A_)).tex())

def norm(M: Matrix) -> float:
    return max([sum([abs(M[i][j]) for i in range(n)]) for j in range(n)])

print_tex(r'||A||_1 * ||A^{-1}||_1 = ', str(norm(A) * norm(A_)))


$$ A^{-1} =  \begin{pmatrix} 0.223813 & -1.273561 & 0.628324 & 0.626037 \\ 0.005509 & 0.612143 & -0.396776 & -0.266127 \\ -0.020830 & 0.216308 & -0.020037 & -0.149376 \\ -0.066053 & -0.432311 & 0.331342 & 0.315062 \end{pmatrix} $$

$$ A \times A^{-1} =  \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

$$ ||A||_1 * ||A^{-1}||_1 =  67.39752702142785 $$